In [ ]:
!pip install llama-index-readers-file pymupdf
!pip install llama-index-vector-stores-postgres
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-llama-cpp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 M

In [ ]:
!pip install chromadb pypdf
!pip install llama-index-vector-stores-chroma

In [ ]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from llama_index.llms.llama_cpp import LlamaCPP

#model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"
model_url = "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q6_K.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=4048,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=8000,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

total size (MB): 5942.06


5667it [00:49, 113.88it/s]                          
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /tmp/llama_index/models/mistral-7b-instruct-v0.1.Q6_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32 

In [ ]:
from pypdf import PdfReader

def load_pdf(file_path):
    """
    Reads the text content from a PDF file and returns it as a single string.

    Parameters:
    - file_path (str): The file path to the PDF file.

    Returns:
    - str: The concatenated text content of all pages in the PDF.

    Raises:
    - FileNotFoundError: If the specified file_path does not exist.
    - PyPDF2.utils.PdfReadError: If the PDF file is encrypted or malformed.

    Example:
    >>> pdf_text = load_pdf("example.pdf")
    >>> print(pdf_text)
    "This is the text content extracted from the PDF file."
    """
    # Logic to read pdf
    reader = PdfReader(file_path)

    # Loop over each page and store it in a variable
    text = ""
    for page in reader.pages:
        text += page.extract_text()

    return text


In [ ]:
pdf_text = load_pdf(file_path="/content/20221_136315_OS_Topic10_Report.pdf")

In [ ]:
import re
def split_text(text: str):
    """
    Splits a text string into a list of non-empty substrings based on the specified pattern.
    The "\n \n" pattern will split the document para by para
    Parameters:
    - text (str): The input text to be split.

    Returns:
    - List[str]: A list containing non-empty substrings obtained by splitting the input text.

    """
    split_text = re.split('\n \n', text)
    return [i for i in split_text if i != ""]

In [ ]:
text = split_text(pdf_text)

In [ ]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore



In [ ]:
# load embeddings from disk

db = chromadb.PersistentClient(path="/content/database")

chroma_collection = db.get_or_create_collection(name = "DB_collection")

for i, d in enumerate(text):
        chroma_collection.add(documents=d, ids=str(i))


vector_store = ChromaVectorStore(chroma_collection=chroma_collection)


In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core.storage.storage_context import StorageContext
from llama_index.core import SimpleDirectoryReader

storage_context = StorageContext.from_defaults(vector_store=vector_store)

reader = SimpleDirectoryReader(input_files=['/content/20221_136315_OS_Topic10_Report.pdf'] )
documents = reader.load_data()

index = VectorStoreIndex.from_documents(
    documents=documents,
    storage_context=storage_context,
    embed_model=embed_model,
    llm=llm,
)

In [ ]:

# Response synthesis
query_engine = index.as_query_engine(llm=llm)

response = query_engine.query("What is the Resource Hierarchy Solution?")

print(response)

llama_perf_context_print:        load time =  511390.50 ms
llama_perf_context_print: prompt eval time =  511389.66 ms /   856 tokens (  597.42 ms per token,     1.67 tokens per second)
llama_perf_context_print:        eval time =  175551.10 ms /   172 runs   ( 1020.65 ms per token,     0.98 tokens per second)
llama_perf_context_print:       total time =  687078.78 ms /  1028 tokens



The Resource Hierarchy Solution is a solution proposed originally by Dijkstra to solve the Dining Philosopher problem. It involves indexing the resources (forks) from 1 to 5, with each philosopher picking up the lower numbered fork first and then the higher numbered fork. The order in which each philosopher puts down the left or right fork first does not matter. If four of the five philosophers simultaneously pick up their lower numbered forks, only the highest numbered fork will remain on the table, so the fifth philosopher will not be able to pick up any fork. This design guarantees deadlock-free. However, the Resource Hierarchy Solution has significant drawbacks, including the requirement for the list of resources to be completely known in advance, lack of fairness, and inefficiency.
